In [1]:
import torch
from diffusers.utils import load_image, make_image_grid
from diffusers import AutoPipelineForInpainting
from PIL import Image, ImageDraw
import random

In [2]:
def generate_random_mask(image_size):
    mask = Image.new("L", image_size, 0)  # Create a blank (black) mask
    draw = ImageDraw.Draw(mask)
    width, height = image_size
    
    x1, y1 = width//4 + random.randint(-width//10, width//10), height//4 + random.randint(-height//10, height//10)
    x2, y2 = 3*width//4 + random.randint(-width//10, width//10), 3*height//4 + random.randint(-height//10, height//10)
    # Draw a filled rectangle
    draw.rectangle([x1, y1, x2, y2], fill=255)
    return mask, (x1, y1, x2, y2)

Testing every checkpoints of LoRA trained model (`fine_tune_lora.sh`)

In [ ]:
NUM_CHECKPOINTS = 1
folders_temp = [f"checkpoint-{500*k}" for k in range(1, NUM_CHECKPOINTS+1)]
folders = ["results20250111183706/" + folder + "/" for folder in folders_temp]

for j, folder in enumerate(folders):
    print(f"{folders_temp[j]}.png")

    pipeline = AutoPipelineForInpainting.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16, variant="fp16"
    )

    # LoRA part

    pipeline.load_lora_weights(folder, weight_name="LoRA_2_pytorch_lora_weights.safetensors")

    #

    pipeline.enable_model_cpu_offload()
    image_grid = []

    for i in range(4):
        # load base and mask image
        init_image = load_image(f"bg_images/back{i+1}.jpg")
        
        # resize them 512x512
        init_image = init_image.resize((512, 512))
        mask_image, mask_coords = generate_random_mask(init_image.size)
        mask_image = mask_image.resize((512, 512))

        generator = torch.Generator("cuda").manual_seed(92)
        prompt = "a piece of trash floating underwater, realistic, blend with the environment"
        #negative_prompt = "text, clean, clear, nothing"
        #prompt = "a plastic trash in the sea, a weathered plastic trash partially buried in the seabed, blending seamlessly with the sand and rocks"
        
        #prompt = "A piece of trash underwater, realistic, highly detailed, blending with the environment"
        negative_prompt = "Unrealistic, clean, blurry"
        
        image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=init_image, mask_image=mask_image, strength=1, generator=generator, padding_mask_crop=32).images[0]
        draw = ImageDraw.Draw(image)
        draw.rectangle(mask_coords, outline="red", width=3)
        image_grid.append(make_image_grid([init_image, mask_image, image], rows=1, cols=3))
    image_grid = make_image_grid(image_grid, rows=4, cols=1)
    image_grid.save(f"inpainting{folders_temp[j]}.png")

    print(f"Saved inpainting{folders_temp[j]}.png")

In [ ]:
NUM_CHECKPOINTS = 2
folders_temp = [f"checkpoint-{500*k}" for k in range(1, NUM_CHECKPOINTS+1)]
folders = ["results20250111183706/" + folder + "/" for folder in folders_temp]

for j, folder in enumerate(folders):
    print(f"{folders_temp[j]}.png")

    pipeline = AutoPipelineForInpainting.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16, variant="fp16"
    )

    # LoRA part

    pipeline.load_lora_weights(folder, weight_name="LoRA_2_pytorch_lora_weights.safetensors")

    #

    pipeline.enable_model_cpu_offload()
    image_grid = []

    for i in range(4):
        # load base and mask image
        init_image = load_image(f"bg_images/back{i+1}.jpg")
        
        # resize them 512x512
        init_image = init_image.resize((512, 512))
        mask_image, mask_coords = generate_random_mask(init_image.size)
        mask_image = mask_image.resize((512, 512))

        generator = torch.Generator("cuda").manual_seed(92)
        prompt = "a piece of trash floating underwater, realistic, blend with the environment"
        #negative_prompt = "text, clean, clear, nothing"
        #prompt = "a plastic trash in the sea, a weathered plastic trash partially buried in the seabed, blending seamlessly with the sand and rocks"
        
        #prompt = "A piece of trash underwater, realistic, highly detailed, blending with the environment"
        negative_prompt = "Unrealistic, clean, blurry"
        
        image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=init_image, mask_image=mask_image, strength=1, generator=generator, padding_mask_crop=32).images[0]
        draw = ImageDraw.Draw(image)
        draw.rectangle(mask_coords, outline="red", width=3)
        image_grid.append(make_image_grid([init_image, mask_image, image], rows=1, cols=3))
    image_grid = make_image_grid(image_grid, rows=4, cols=1)
    image_grid.save(f"inpainting{folders_temp[j]}.png")

    print(f"Saved inpainting{folders_temp[j]}.png")

In [ ]:
import peft
print(peft.__version__)

Testing with Dreambooth trained model (`fine_tune_dreambooth_inpaint.sh`)

In [ ]:
pipeline = AutoPipelineForInpainting.from_pretrained(
    "results/results20250111192740", torch_dtype=torch.float16
)

pipeline.enable_model_cpu_offload()
image_grid = []

for i in range(4):
    # load base and mask image
    init_image = load_image(f"bg_images/back{i+1}.jpg")
    
    # resize them 512x512
    init_image = init_image.resize((512, 512))
    mask_image, mask_coords = generate_random_mask(init_image.size)
    mask_image = mask_image.resize((512, 512))

    generator = torch.Generator("cuda").manual_seed(92)
    prompt = "a piece of trash floating underwater, realistic, blend with the environment"
    #negative_prompt = "text, clean, clear, nothing"
    #prompt = "a plastic trash in the sea, a weathered plastic trash partially buried in the seabed, blending seamlessly with the sand and rocks"
    
    #prompt = "A piece of trash underwater, realistic, highly detailed, blending with the environment"
    negative_prompt = "Unrealistic, clean, blurry"
    
    image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=init_image, mask_image=mask_image, strength=1, generator=generator, padding_mask_crop=32).images[0]
    draw = ImageDraw.Draw(image)
    draw.rectangle(mask_coords, outline="red", width=3)
    image_grid.append(make_image_grid([init_image, mask_image, image], rows=1, cols=3))
image_grid = make_image_grid(image_grid, rows=4, cols=1)
image_grid.save(f"inpainting.png")
print(f"Saved inpainting.png")